In [1]:
pip install customtkinter pillow

                                              0.0/296.1 kB ? eta -:--:--
     -----                                   41.0/296.1 kB 2.0 MB/s eta 0:00:01
     -------------                          102.4/296.1 kB 1.2 MB/s eta 0:00:01
     ----------------------------           225.3/296.1 kB 1.7 MB/s eta 0:00:01
     ------------------------------------   286.7/296.1 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 296.1/296.1 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import customtkinter as ctk
import cv2
import mediapipe as mp
import numpy as np
import pickle
from PIL import Image
import threading
import os
import gc
from pygame import mixer
from gtts import gTTS
import sys
import time

# --- KONFIGURASI TEMA ---
ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")

class SignTalkApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        # --- BRANDING ---
        self.title("SignTalk AI - By Afrido Ganteng")
        self.geometry("1200x850")
        
        # Color Palettes
        self.clr_bg = ("#F5F5F5", "#050505")
        self.clr_sidebar = ("#E0E0E0", "#0D0D0D")
        self.clr_text = ("#000000", "#FFFFFF")
        self.clr_accent = "#00CCFF"
        self.clr_card = ("#FFFFFF", "#121212")

        self.configure(fg_color=self.clr_bg)

        # 1. ENGINE LOAD
        mixer.init()
        self.load_model()
        self.init_mediapipe()

        # 2. STATE CONTROL (DIPERKETAT UNTUK STABILITAS)
        self.cap = None
        self.is_running = False
        self._img_ref = None 
        self.camera_lock = threading.Lock() # Kunci Pengaman Thread
        
        self.speed_levels = ["Lambat", "Medium", "Cepat", "Instan"]
        self.speed_values = [25, 12, 5, 1] 
        self.stability_threshold = 12 
        
        self.prediction_history = [] 
        self.last_confirmed_word = ""
        self.history_sentence = []

        # 3. SETUP UI
        self.setup_ui()
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def load_model(self):
        try:
            model_path = 'models/model_sign_language.pkl'
            if os.path.exists(model_path):
                with open(model_path, 'rb') as f:
                    self.model = pickle.load(f)
        except: self.model = None

    def init_mediapipe(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(max_num_hands=2, model_complexity=0, min_detection_confidence=0.5)
        self.mp_drawing = mp.solutions.drawing_utils

    def setup_ui(self):
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # --- SIDEBAR ---
        self.sidebar = ctk.CTkFrame(self, width=260, corner_radius=0, fg_color=self.clr_sidebar)
        self.sidebar.grid(row=0, column=0, sticky="nsew")

        self.logo = ctk.CTkLabel(self.sidebar, text="SignTalk AI", font=ctk.CTkFont(size=28, weight="bold"), text_color=self.clr_accent)
        self.logo.pack(pady=(40, 5))
        
        self.ver_label = ctk.CTkLabel(self.sidebar, text="Version 2.3 Stable", font=ctk.CTkFont(size=12), text_color=("#888888", "#555555"))
        self.ver_label.pack(pady=(0, 30))

        self.btn_start = ctk.CTkButton(self.sidebar, text="ACTIVATE AI", height=50, fg_color=self.clr_accent, text_color="black", font=ctk.CTkFont(weight="bold"), command=self.toggle_camera)
        self.btn_start.pack(pady=10, padx=30)

        self.btn_reset = ctk.CTkButton(self.sidebar, text="RESET KALIMAT", height=45, fg_color="transparent", border_width=2, border_color=self.clr_accent, text_color=self.clr_text, command=self.clear_history)
        self.btn_reset.pack(pady=10, padx=30)

        # SLIDER KECEPATAN
        self.speed_lbl_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        self.speed_lbl_frame.pack(pady=(40, 0), padx=30, fill="x")
        ctk.CTkLabel(self.speed_lbl_frame, text="Respon:", font=ctk.CTkFont(size=12, weight="bold"), text_color=self.clr_text).pack(side="left")
        self.speed_val_show = ctk.CTkLabel(self.speed_lbl_frame, text="Medium", font=ctk.CTkFont(size=12), text_color=self.clr_accent)
        self.speed_val_show.pack(side="right")

        self.speed_slider = ctk.CTkSlider(self.sidebar, from_=0, to=3, number_of_steps=3, command=self.slider_event)
        self.speed_slider.set(1) 
        self.speed_slider.pack(pady=(10, 5), padx=30, fill="x")

        self.theme_menu = ctk.CTkOptionMenu(self.sidebar, values=["Dark", "Light"], command=ctk.set_appearance_mode)
        self.theme_menu.pack(side="bottom", pady=30, padx=30)

        # --- MAIN AREA ---
        self.main_area = ctk.CTkFrame(self, corner_radius=25, fg_color=self.clr_bg)
        self.main_area.grid(row=0, column=1, padx=20, pady=20, sticky="nsew")

        self.sentence_display = ctk.CTkLabel(self.main_area, text="Mulai berisyarat...", font=ctk.CTkFont(size=24, weight="bold"), text_color=self.clr_text, wraplength=800)
        self.sentence_display.pack(pady=(30, 10), padx=40)

        self.video_container = ctk.CTkFrame(self.main_area, fg_color="transparent")
        self.video_container.pack(expand=True, fill="both", padx=10, pady=10)

        self.video_label = ctk.CTkLabel(self.video_container, text="SYSTEM READY", font=("Arial", 20), text_color=("#AAAAAA", "#444444"))
        self.video_label.pack(expand=True, fill="both")

        # CIRCULAR BUFFER (Canvas)
        self.canvas_buffer = ctk.CTkCanvas(self.video_container, width=100, height=100, bg=self.clr_bg[1] if ctk.get_appearance_mode()=="Dark" else self.clr_bg[0], highlightthickness=0)
        self.angle = 0

        # Caption Bar
        self.caption_container = ctk.CTkFrame(self.main_area, height=140, corner_radius=25, fg_color=self.clr_card, border_width=1, border_color=("#CCCCCC", "#222222"))
        self.caption_container.pack(side="bottom", fill="x", padx=40, pady=(0, 40))
        self.caption_container.pack_propagate(False) 

        self.text_result = ctk.CTkLabel(self.caption_container, text="...", font=ctk.CTkFont(size=75, weight="bold"), text_color=self.clr_text)
        self.text_result.place(relx=0.5, rely=0.5, anchor="center")

    # --- LOGIKA STABILITAS TINGGI ---
    def rotate_buffer(self):
        if self.is_running or not self.canvas_buffer.winfo_viewable(): return
        self.canvas_buffer.delete("all")
        self.angle = (self.angle + 12) % 360
        self.canvas_buffer.create_arc(10, 10, 90, 90, start=self.angle, extent=120, outline=self.clr_accent, width=5, style="arc")
        self.after(15, self.rotate_buffer)

    def toggle_camera(self):
        if not self.is_running:
            self.video_label.pack_forget()
            self.canvas_buffer.place(relx=0.5, rely=0.5, anchor="center")
            self.rotate_buffer()
            self.btn_start.configure(state="disabled", text="WAIT...")
            
            # Gunakan Thread agar tidak mengunci UI
            threading.Thread(target=self.safe_start, daemon=True).start()
        else:
            self.stop_system()

    def safe_start(self):
        with self.camera_lock: # Pastikan tidak ada thread lain yang sedang stop/start
            # 1. Pastikan Kamera benar-benar kosong
            if self.cap:
                self.cap.release()
                time.sleep(0.5)

            # 2. Buka Kamera
            self.cap = cv2.VideoCapture(0)
            if self.cap.isOpened():
                # 3. Sinkronisasi UI
                self.after(0, self.finalize_start)
            else:
                self.after(0, self.loading_failed)

    def finalize_start(self):
        self.canvas_buffer.place_forget()
        self.video_label.pack(expand=True, fill="both")
        self.is_running = True
        self.btn_start.configure(state="normal", text="STOP SYSTEM", fg_color="#FF3333")
        threading.Thread(target=self.video_worker, daemon=True).start()

    def stop_system(self):
        self.is_running = False
        self.btn_start.configure(state="disabled", text="STOPPING...")
        
        def _cleanup():
            with self.camera_lock:
                time.sleep(0.3) # Jeda agar worker thread berhenti dulu
                if self.cap:
                    self.cap.release()
                    self.cap = None
                
                # Reset UI ke posisi awal
                self.after(0, self.finalize_stop)

        threading.Thread(target=_cleanup, daemon=True).start()

    def finalize_stop(self):
        self.video_label.configure(image=None, text="SYSTEM READY")
        self._img_ref = None
        self.btn_start.configure(state="normal", text="ACTIVATE AI", fg_color=self.clr_accent)
        gc.collect()

    def loading_failed(self):
        self.canvas_buffer.place_forget()
        self.video_label.pack(expand=True, fill="both")
        self.video_label.configure(text="KAMERA GAGAL DIBUKA")
        self.btn_start.configure(state="normal", text="ACTIVATE AI", fg_color=self.clr_accent)

    def video_worker(self):
        # Tambahkan pengaman cap is not None
        while self.is_running and self.cap:
            ret, frame = self.cap.read()
            if not ret: break
            
            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = self.hands.process(rgb)
            
            if results.multi_hand_landmarks and self.model:
                data_row = np.zeros(126).tolist()
                for idx, hl in enumerate(results.multi_hand_landmarks):
                    if idx > 1: break
                    self.mp_drawing.draw_landmarks(frame, hl, self.mp_hands.HAND_CONNECTIONS)
                    for i, lm in enumerate(hl.landmark):
                        start = (idx * 63) + (i * 3)
                        data_row[start:start+3] = [lm.x, lm.y, lm.z]
                
                try:
                    pred = self.model.predict([data_row])[0].upper()
                    self.prediction_history.append(pred)
                    if len(self.prediction_history) > (self.stability_threshold + 5):
                        self.prediction_history.pop(0)

                    most_freq = max(set(self.prediction_history), key=self.prediction_history.count)
                    if self.prediction_history.count(most_freq) >= self.stability_threshold:
                        if most_freq != self.last_confirmed_word:
                            self.after(0, self.update_text, most_freq)
                            self.last_confirmed_word = most_freq
                except: pass

            # Update Frame
            img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            self._img_ref = ctk.CTkImage(light_image=img_pil, dark_image=img_pil, size=(850, 520))
            self.after(0, self.update_image)
            time.sleep(0.005)

    def update_text(self, word):
        if not self.is_running: return
        self.text_result.configure(text=word)
        self.history_sentence.append(word)
        if len(self.history_sentence) > 8: self.history_sentence.pop(0)
        self.sentence_display.configure(text=" ".join(self.history_sentence))
        self.speak(word)

    def update_image(self):
        if self.is_running and self.video_label.winfo_exists():
            self.video_label.configure(image=self._img_ref)

    def slider_event(self, value):
        idx = int(value)
        self.stability_threshold = self.speed_values[idx]
        self.speed_val_show.configure(text=self.speed_levels[idx])
        self.prediction_history = [] 
        self.last_confirmed_word = ""

    def clear_history(self):
        self.history_sentence = []
        self.sentence_display.configure(text="Mulai berisyarat...")
        self.text_result.configure(text="...")
        self.last_confirmed_word = ""
        self.prediction_history = []

    def speak(self, text):
        def _run():
            try:
                if not os.path.exists('assets'): os.makedirs('assets')
                path = f"assets/{text}.mp3"
                if not os.path.exists(path): gTTS(text, lang='id').save(path)
                mixer.music.load(path)
                mixer.music.play()
            except: pass
        threading.Thread(target=_run, daemon=True).start()

    def on_closing(self):
        self.is_running = False
        time.sleep(0.2)
        if self.cap: self.cap.release()
        self.destroy()
        sys.exit(0)

if __name__ == "__main__":
    app = SignTalkApp()
    app.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
D:\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifi

SystemExit: 0

D:\anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
